In [4]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Load the data
file_path = 'C:/Users/olw09/karthikcase/descrete-optimization/food/nnn.xlsx'
df = pd.read_excel(file_path)

dishes = df['Dishes']
p = pd.to_numeric(df['Profit(₹)'], errors='coerce').fillna(0).values
m = pd.to_numeric(df['Loss_on_demand(₹)'], errors='coerce').fillna(0).values
l = pd.to_numeric(df['Loss_on_wastage₹'], errors='coerce').fillna(0).values
L = pd.to_numeric(df['shelf_life(hr)'], errors='coerce').fillna(0).values

num_dishes = len(dishes)
num_hours = 14

np.random.seed(0)  
requirement = np.zeros((num_dishes, num_hours), dtype=int)
requirement = np.random.randint(0, 11, size=(num_dishes, num_hours))

#print("Requirement Data:")
#print(requirement)

model = gp.Model("restaurant_optimization")

prepare = model.addVars(num_dishes, num_hours, vtype=GRB.INTEGER, lb=0, name="Prepare")
Inventory = model.addVars(num_dishes, num_hours, vtype=GRB.INTEGER, lb=0, name="Inventory")
waste = model.addVars(num_dishes, num_hours, vtype=GRB.INTEGER, lb=0, name="waste")
unfull = model.addVars(num_dishes, num_hours, vtype=GRB.INTEGER, lb=0, name="unfill")
sold = model.addVars(num_dishes, num_hours, vtype=GRB.INTEGER, lb=0, name="sold")

profit = 0
losswaste = 0
losteunfill = 0

for i in range(num_dishes):
    for h in range(num_hours):
        dummy = model.addVar(lb=0, vtype=GRB.INTEGER, name=f'dish{i} hour{h}')
        model.addConstr(prepare[i, h] == 10 * dummy)
        if h < L[i]:
            model.addConstr(waste[i, h] == 0)
        model.addConstr(sold[i, h] <= requirement[i, h] )
        model.addConstr(unfull[i,h]>=requirement[i,h]-sold[i,h])
        model.addConstr(unfull[i,h]>=-requirement[i,h]+sold[i,h])
        model.addConstr(sold[i, h] <= prepare[i, h] + Inventory[i, h])
        model.addConstr(Inventory[i, h] == sum(prepare[i, t] - sold[i, t] - waste[i, t] for t in range(h + 1)))
        if h >= L[i]:
            model.addConstr(waste[i, h] + waste[i, h - 1] == sum(Inventory[i, t] for t in range(h - L[i]+1)))

        profit += p[i] * sold[i, h]
        losswaste += l[i] * waste[i, h]
        losteunfill += m[i] * unfull[i, h]

model.setObjective(profit - losswaste - losteunfill, GRB.MAXIMIZE)
model.optimize()

result_data = []
for i in range(num_dishes):
    for h in range(num_hours):
        result_data.append({
            'Dish': dishes[i],
            'Hour': h,
            'Requirement': requirement[i,h],
            'Prepare': prepare[i, h].X,
            'Inventory': Inventory[i, h].X,
            'Waste': waste[i, h].X,
            'Unfilled': unfull[i, h].X,
            'Sold': sold[i, h].X
        })

results_df = pd.DataFrame(result_data)
print("Optimization Results:")
print(results_df)

results_df.to_excel('optimization_results.xlsx', index=False)



Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1960 rows, 1680 columns and 10877 nonzeros
Model fingerprint: 0xcb351826
Variable types: 0 continuous, 1680 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective -20010.00000
Presolve removed 1927 rows and 1633 columns
Presolve time: 0.57s
Presolved: 33 rows, 47 columns, 217 nonzeros
Found heuristic solution: objective 8247.0000000
Variable types: 0 continuous, 47 integer (1 binary)

Root relaxation: objective 1.021654e+04, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.csv")
df

,Dish,Hour,Requirement,Prepare,Inventory,Waste,Unfilled,Sold
0,Samosa,5,5,10.0,0.0,0.0,0.0,5.0
1,Samosa,6,0,0.0,5.0,0.0,0.0,0.0
2,Samosa,7,3,0.0,5.0,0.0,0.0,3.0
3,Samosa,8,3,10.0,2.0,0.0,0.0,3.0
4,Samosa,9,7,0.0,9.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...
175,ice cream,9,5,0.0,6.0,1.0,0.0,5.0
176,ice cream,10,3,10.0,-0.0,1.0,0.0,3.0
177,ice cream,11,1,0.0,6.0,3.0,0.0,1.0
178,ice cream,12,8,20.0,2.0,7.0,0.0,8.0
